In [1]:
import nltk
import pickle
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from  sklearn.decomposition import NMF
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.patheffects as PathEffects
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from textblob import TextBlob
from sklearn.preprocessing import normalize
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/yefeng/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
with open('data/filtered_sent_sentiment_score.pkl', 'rb') as picklefile: 
    df_nlp = pickle.load(picklefile)

In [3]:
df_nlp.head(3)

,rev_id,sentence,lem_sent_token,lem_sent_text,0,1,2,3,4,5,...,13,14,15,16,17,18,19,polarity,subjectivity,sent_score
0,0,I'm glad our family experienced Squaw but I wo...,"[i, 'm, glad, our, family, experience, squaw, ...",i 'm glad our family experience squaw but i wo...,0.00000,0.00000,0.000000,0.000000,0.438666,0.00000,...,0.001622,0.000000,0.009287,0.000000,0.0,0.000000,0.515256,0.433333,0.633333,0.274444
1,0,Here is a list of cons in detail: \n1.,"[here, be, a, list, of, con, in, detail, :, 1, .]",here be a list of con in detail : 1 .,0.04973,0.14709,0.002256,0.008267,0.206246,0.10508,...,0.068413,0.000000,0.000000,0.136429,0.0,0.037268,0.000196,0.000000,0.000000,0.000000
2,0,The staff were moody which is odd for an expen...,"[the, staff, be, moody, which, be, odd, for, a...",the staff be moody which be odd for an expensi...,0.00000,0.00000,0.000000,0.000000,0.000000,0.00000,...,0.000000,0.544505,0.000000,0.067645,0.0,0.000000,0.000000,-0.333333,0.475000,-0.158333


In [4]:
with open('data/filtered_sent_lemm.pkl', 'rb') as picklefile: 
    df= pickle.load(picklefile)

In [5]:
df.head()

,rev_id,sentence,lem_sent_token,lem_sent_text
0,0,I'm glad our family experienced Squaw but I wo...,"[i, 'm, glad, our, family, experience, squaw, ...",i 'm glad our family experience squaw but i wo...
1,0,Here is a list of cons in detail: \n1.,"[here, be, a, list, of, con, in, detail, :, 1, .]",here be a list of con in detail : 1 .
2,0,The staff were moody which is odd for an expen...,"[the, staff, be, moody, which, be, odd, for, a...",the staff be moody which be odd for an expensi...
3,0,2.,"[2, .]",2 .
4,0,The on hill ski staff had the worst ego's.,"[the, on, hill, ski, staff, have, the, bad, eg...",the on hill ski staff have the bad ego 's .


In [6]:
for sent_i in np.argsort(df_nlp.loc[:,5])[-5:]:
    
    print(df_nlp.iloc[sent_i,1],": ", df_nlp.iloc[sent_i,26])
    print("-"*30)

Lots of fun on the slopes. :  0.06
------------------------------
Lots to do. :  0.0
------------------------------
It's a lot safer. :  0.0
------------------------------
Ski there lots. :  0.0
------------------------------
The park is a lot of fun. :  0.06
------------------------------


In [7]:
# Topic:
# beginner:6
# food: 15
# family: 19
# crowded: 0
# liftspeed: 1
# waittime: 2
# groom: 18
# powder: 17
# staff: 14
# terrain: 7

df_nlp.columns

Index([        'rev_id',       'sentence', 'lem_sent_token',  'lem_sent_text',
                      0,                1,                2,                3,
                      4,                5,                6,                7,
                      8,                9,               10,               11,
                     12,               13,               14,               15,
                     16,               17,               18,               19,
             'polarity',   'subjectivity',     'sent_score'],
      dtype='object')

In [10]:
df_out = df_nlp[['rev_id','sentence','polarity','subjectivity', 'sent_score',0,1,2,6,7,14,15,17,18,19]].copy()
df_out.rename(columns={0:'crowded',1:'liftspeed',2:'waittime',6:'beginner',7:'terrain',14:'staff',
                       15:'food',17:'powder',18:'groom',19:'family'},inplace=True)

In [11]:
filters = ['staff','terrain','waittime','liftspeed','powder','groom','family','beginner','food','crowded']

for i in filters:
    df_out[i]=df_out[i]*df_out['sent_score']

In [12]:
for sent_i in np.argsort(df_out.loc[:,'food'])[-10:]:
    
    print(df_out.iloc[sent_i,1],": ", df_out.iloc[sent_i,4])
    print("-"*30)

The area was beautiful. :  0.85
------------------------------
Lower area is ideal for beginners/novices. :  0.9
------------------------------
Lower area is ideal for beginners/novices. :  0.9
------------------------------
An excellent Michigan ski area. :  1.0
------------------------------
An excellent Michigan ski area. :  1.0
------------------------------
Excellent Ski Area! :  1.0
------------------------------
Tim and Diane Mueller have created a wonderful, expansive ski area. :  1.0
------------------------------
Mammoth is one of the greatest ski areas out there. :  1.0
------------------------------
Lutsen to me is the greatest ski area in Minnesota, Wisconsin, and the Dakotas. :  1.0
------------------------------
With that soapbox speech over, Mammoth is one of the U.S.'s greatest ski areas. :  1.0
------------------------------


In [13]:
df_out = df_out[['rev_id','sentence','staff','terrain','waittime','liftspeed','powder','groom','family','beginner','food','crowded']]

In [19]:
df_out.head()

,rev_id,sentence,staff,terrain,waittime,liftspeed,powder,groom,family,beginner,food,crowded
0,0,I'm glad our family experienced Squaw but I wo...,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.141409,0.000000,0.002549,0.0
1,0,Here is a list of cons in detail: \n1.,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0
2,0,The staff were moody which is odd for an expen...,-0.086213,-0.0,-0.000000,-0.0,-0.0,-0.000000,-0.000000,-0.000000,-0.000000,-0.0
3,0,2.,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0
4,0,The on hill ski staff had the worst ego's.,-0.604578,-0.0,-0.010784,-0.0,-0.0,-0.022008,-0.000000,-0.178935,-0.000000,-0.0


In [25]:
for f in filters:
    df_out[f] = df_out[f].fillna(0)
                       


In [26]:
df_out = df_out.groupby('rev_id')[filters].mean()

In [27]:
df_out

,staff,terrain,waittime,liftspeed,powder,groom,family,beginner,food,crowded
rev_id,,,,,,,,,,
0,-0.030494,0.007056,0.001051,0.001554,0.000028,-0.000656,0.007617,-0.002121,-0.001750,0.000266
1,0.007892,0.001958,0.000000,0.001208,0.003402,0.000831,0.000839,-0.000071,0.007967,-0.018648
2,0.000000,-0.000440,0.000000,-0.085200,-0.057809,0.000000,-0.228727,0.000000,0.000000,0.000000
3,0.000906,0.003773,0.001293,0.001821,0.001330,0.000671,-0.000009,0.000634,0.001782,0.003239
4,0.000789,0.131268,-0.011292,0.034590,0.002627,0.001151,-0.000056,0.000968,0.005363,0.000478
...,...,...,...,...,...,...,...,...,...,...
18257,0.000000,0.000416,0.000000,-0.006370,0.000617,0.000269,-0.000096,0.001754,0.002327,0.003535
18258,0.000000,0.000643,0.000000,0.001216,0.000246,0.000000,0.000000,0.000000,0.009821,0.000250
18259,0.005281,0.012190,0.000651,-0.010659,0.004833,-0.000034,0.000346,0.001154,0.009056,0.014482


In [28]:
df = pd.read_csv('data/OnTheSnow_SkiAreaReviews.csv')

In [32]:
df_full = df.join(df_out, how='inner')

In [34]:
with open('data/topics_recommender.pkl', 'wb') as picklefile:
    pickle.dump(df_full, picklefile)

In [37]:
df_full.columns

Index(['State', 'Ski Area', 'Reviewer Name', 'Review Date',
       'Review Star Rating (out of 5)', 'Review Text', 'staff', 'terrain',
       'waittime', 'liftspeed', 'powder', 'groom', 'family', 'beginner',
       'food', 'crowded'],
      dtype='object')